# Modeling for garbage classification

## Model VGG16

For the garbage classification, we choose to classify the garbage with the VGG16 model which is a famous CNN model used for image classification

In [1]:
%load_ext autoreload
%autoreload 2

### Data

In [2]:
%pwd

'/Users/walkyz/code/kirianpg/waste_sorter_smart_bin/notebooks'

In [3]:
import tensorflow as tf
print(tf.__version__)

2.10.0


In [4]:
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import numpy as np
import os
from PIL import Image

def load_flowers_data():
    data_path = '../data/raw_data/kaggle_data/Garbage classification/Garbage classification'
    classes = {
        'trash': 0,
        'glass': 1,
        'paper': 2,
        'cardboard': 3,
        'plastic': 4,
        'metal': 5}
    
    imgs = []
    labels = []
    for (cl, i) in classes.items():
        images_path = [elt for elt in os.listdir(os.path.join(data_path, cl)) if elt.find('.jpg')>0]
        for img in tqdm(images_path[:300]):
            path = os.path.join(data_path, cl, img)
            if os.path.exists(path):
                image = Image.open(path)
                image = image.resize((224, 224))
                imgs.append(np.array(image))
                labels.append(i)

    X = np.array(imgs)
    num_classes = len(set(labels))
    y = to_categorical(labels, num_classes)

    # Finally we shuffle:
    p = np.random.permutation(len(X))
    X, y = X[p], y[p]

    first_split = int(len(imgs) /6.)
    second_split = first_split + int(len(imgs) * 0.2)
    X_test, X_val, X_train = X[:first_split], X[first_split:second_split], X[second_split:]
    y_test, y_val, y_train = y[:first_split], y[first_split:second_split], y[second_split:]

    return X_train, y_train, X_val, y_val, X_test, y_test, num_classes

In [5]:
# CALL load_flowers_data WITH YOUR PREFERRED METHOD HERE
X_train, y_train, X_val, y_val, X_test, y_test, num_classes = load_flowers_data()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 365.74it/s]


In [6]:
X_train

array([[[[240, 225, 204],
         [240, 225, 204],
         [240, 225, 204],
         ...,
         [232, 217, 198],
         [232, 217, 198],
         [232, 217, 198]],

        [[240, 225, 204],
         [240, 225, 204],
         [240, 225, 204],
         ...,
         [232, 217, 198],
         [232, 217, 198],
         [232, 217, 198]],

        [[240, 225, 204],
         [240, 225, 204],
         [240, 225, 204],
         ...,
         [232, 217, 198],
         [232, 217, 198],
         [232, 217, 198]],

        ...,

        [[238, 225, 206],
         [238, 225, 206],
         [238, 225, 206],
         ...,
         [ 83,  74,  59],
         [ 83,  74,  59],
         [ 83,  74,  59]],

        [[238, 225, 206],
         [238, 225, 206],
         [238, 225, 206],
         ...,
         [ 83,  74,  59],
         [ 83,  74,  59],
         [ 83,  74,  59]],

        [[238, 225, 206],
         [238, 225, 206],
         [238, 225, 206],
         ...,
         [ 83,  74,  59],
        

In [26]:
y_val

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]], dtype=float32)

### Initialize the model

In [7]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models, layers, optimizers, regularizers
from tensorflow.keras.models import Sequential

In [8]:
def load_model_VGG16():

    model = VGG16(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
    )

    return model

In [9]:
model_VGG16 = load_model_VGG16()

In [10]:
model_VGG16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

### Lock the layers of the VGG16 model

We need to preserve the training of the initial model

In [11]:
def set_nontrainable_layers(model):

    model.trainable = False
    
    return model

In [12]:
model_VGG16_snl = set_nontrainable_layers(model_VGG16)

In [13]:
model_VGG16_snl.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

### Transfer learning + regularizers

In [14]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top, + regularizers'''
    reg = regularizers.l1_l2(l2=0.005)
    flattening_layer = layers.Flatten()
    dense_layer = layers.Dense(512, activation='relu', kernel_regularizer=reg)
    dropout_layer = layers.Dropout(0.5)
    prediction_layer = layers.Dense(6, activation='softmax')

    final_model = Sequential([
        model,
        flattening_layer,
        dense_layer,
        dropout_layer,
        prediction_layer
    ]) 
    
    return final_model

In [15]:
customized_VGG16 = add_last_layers(model_VGG16_snl)

In [16]:
customized_VGG16.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 512)               12845568  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 6)                 3078      
                                                                 
Total params: 27,563,334
Trainable params: 12,848,646
Non-trainable params: 14,714,688
_________________________________________________________________


### Compiling the model

In [17]:
def build_model():
  adam = optimizers.Adam(learning_rate=1e-4)
  customized_VGG16.compile(
    loss='categorical_crossentropy',
    optimizer=adam,
    metrics=['accuracy']
  )

  return customized_VGG16

In [18]:
builded_VGG16 = build_model()

In [19]:
builded_VGG16.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 512)               12845568  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 6)                 3078      
                                                                 
Total params: 27,563,334
Trainable params: 12,848,646
Non-trainable params: 14,714,688
_________________________________________________________________


### Process the data

In [20]:
from tensorflow.keras.applications.vgg16 import preprocess_input

In [21]:
X_train_preproc = preprocess_input(X_train)
X_test_preproc = preprocess_input(X_test)
X_val_preproc = preprocess_input(X_val)

In [22]:
X_train_preproc

array([[[[100.061    , 108.221    , 116.32     ],
         [100.061    , 108.221    , 116.32     ],
         [100.061    , 108.221    , 116.32     ],
         ...,
         [ 94.061    , 100.221    , 108.32     ],
         [ 94.061    , 100.221    , 108.32     ],
         [ 94.061    , 100.221    , 108.32     ]],

        [[100.061    , 108.221    , 116.32     ],
         [100.061    , 108.221    , 116.32     ],
         [100.061    , 108.221    , 116.32     ],
         ...,
         [ 94.061    , 100.221    , 108.32     ],
         [ 94.061    , 100.221    , 108.32     ],
         [ 94.061    , 100.221    , 108.32     ]],

        [[100.061    , 108.221    , 116.32     ],
         [100.061    , 108.221    , 116.32     ],
         [100.061    , 108.221    , 116.32     ],
         ...,
         [ 94.061    , 100.221    , 108.32     ],
         [ 94.061    , 100.221    , 108.32     ],
         [ 94.061    , 100.221    , 108.32     ]],

        ...,

        [[102.061    , 108.221    , 11

In [29]:
X_train_preproc.shape

(1038, 224, 224, 3)

In [27]:
X_train_preproc.shape[1:]

(224, 224, 3)

In [24]:
type(X_train_preproc)

numpy.ndarray

### Fit the model

In [23]:
from tensorflow.keras.callbacks import EarlyStopping

In [24]:
es = EarlyStopping(patience=10, restore_best_weights=True)

In [25]:
history = builded_VGG16.fit(
    X_train_preproc, y_train,
    batch_size = 64,
    epochs = 5,
    verbose=1,
    callbacks=[es],
    validation_data=(X_val_preproc, y_val)
)

Epoch 1/5


2024-03-12 19:05:43.312650: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


 1/17 [>.............................] - ETA: 2:04 - loss: 1003.3214 - accuracy: 0.1875

KeyboardInterrupt: 

In [ ]:
history = builded_VGG16.fit(
    X_train_preproc, y_train,
    batch_size = 64,
    epochs = 15,
    verbose=1,
    callbacks=[es],
    validation_data=(X_val_preproc, y_val)
)

### Learning Curves

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_history(history, title='', axs=None, exp_name=""):
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label='train' + exp_name)
    ax1.plot(history.history['val_loss'], label='val' + exp_name)
    #ax1.set_ylim(0., 2.2)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['accuracy'], label='train accuracy'  + exp_name)
    ax2.plot(history.history['val_accuracy'], label='val accuracy'  + exp_name)
    #ax2.set_ylim(0.25, 1.)
    ax2.set_title('Accuracy')
    ax2.legend()
    return (ax1, ax2)

In [ ]:
plot_history(history)

In [ ]:
plot_history(history)

### Evaluate the model

In [ ]:
builded_VGG16.evaluate(X_test_preproc, y_test)

In [ ]:
builded_VGG16.evaluate(X_test_preproc, y_test)

### Make predictions

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

#### Trash predicting

In [ ]:
img_path = '../data/raw_data/kaggle_data/Garbage classification/Garbage classification/trash/trash40.jpg'
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

predictions = builded_VGG16.predict(x)

classes = ['trash', 'glass', 'paper', 'cardboard', 'plastic', 'metal']

plt.imshow(img)
print('\n Prediction :')
for i in range(6):
    print(f'{classes[i]} : {round(float(predictions[0][i]), 2)}')